In [34]:
QUERY_SEED_NUMBER = 10

In [35]:
import numpy as np
DB_SEED_NUMBER = 42
ELEMENT_SIZE = np.dtype(np.float32).itemsize
DIMENSION = 64

In [50]:







import json
import numpy as np
import os
import shutil
import tqdm
import heapq
from sklearn.cluster import KMeans
from typing import Annotated


DB_SEED_NUMBER = 42
ELEMENT_SIZE = np.dtype(np.float32).itemsize
DIMENSION = 64

class VecDB:
    def __init__(self, database_file_path = "saved_db.dat", index_file_path = "index.dat", new_db = True, db_size = None) -> None:
        self.db_path = database_file_path
        self.index_path = index_file_path
        if new_db:
            if db_size is None:
                raise ValueError("You need to provide the size of the database")
            # delete the old DB file if exists
            if os.path.exists(self.db_path):
                os.remove(self.db_path)
            self.generate_database(db_size)
    
    def generate_database(self, size: int) -> None:
        rng = np.random.default_rng(DB_SEED_NUMBER)
        vectors = rng.random((size, DIMENSION), dtype=np.float32)
        self._write_vectors_to_file(vectors)
        self._build_index()

    def _write_vectors_to_file(self, vectors: np.ndarray) -> None:
        mmap_vectors = np.memmap(self.db_path, dtype=np.float32, mode='w+', shape=vectors.shape)
        mmap_vectors[:] = vectors[:]
        mmap_vectors.flush()

    def _get_num_records(self) -> int:
        return os.path.getsize(self.db_path) // (DIMENSION * ELEMENT_SIZE)

    def insert_records(self, rows: Annotated[np.ndarray, (int, 64)]):
        num_old_records = self._get_num_records()
        num_new_records = len(rows)
        full_shape = (num_old_records + num_new_records, DIMENSION)
        mmap_vectors = np.memmap(self.db_path, dtype=np.float32, mode='r+', shape=full_shape)
        mmap_vectors[num_old_records:] = rows
        mmap_vectors.flush()
        #TODO: might change to call insert in the index, if you need
        self._build_index()

    def get_one_row(self, row_num: int) -> np.ndarray:
        # This function is only load one row in memory
        try:
            offset = row_num * DIMENSION * ELEMENT_SIZE
            mmap_vector = np.memmap(self.db_path, dtype=np.float32, mode='r', shape=(1, DIMENSION), offset=offset)
            return np.array(mmap_vector[0])
        except Exception as e:
            return f"An error occurred: {e}"

    def get_all_rows(self) -> np.ndarray:
        # Take care this load all the data in memory
        num_records = self._get_num_records()
        vectors = np.memmap(self.db_path, dtype=np.float32, mode='r', shape=(num_records, DIMENSION))
        return np.array(vectors)
    
    
    
    def _cal_score(self, vec1, vec2):
        dot_product = np.dot(vec1, vec2)
        norm_vec1 = np.linalg.norm(vec1)
        norm_vec2 = np.linalg.norm(vec2)
        cosine_similarity = dot_product / (norm_vec1 * norm_vec2)
        return cosine_similarity



    def retrieve(self, query, top_k=5, n_probe=None, chunk_size=10000):
        query = np.asarray(query, dtype=np.float32).squeeze()

        query_norm = np.linalg.norm(query)
        if query_norm == 0:
            query_norm = 1.0
        normalized_query = query / query_norm

        centers_path = os.path.join(self.index_path, "centers.npy")
        if not os.path.exists(centers_path):
            del query, normalized_query
            return []

        # Load centroids with memory-mapping to save RAM
        centroids = np.load(centers_path, mmap_mode="r")
        n_centroids = centroids.shape[0]

        # Determine number of centroids to probe
        if n_probe is None:
            n_probe = max(1, min(n_centroids, int(np.sqrt(self._get_num_records()))))
            num_records = self._get_num_records()
            if num_records <= 10 * 10**6:
                n_probe = 12
            elif num_records == 15 * 10**6:
                n_probe = 10

        # Compute similarity (cosine) to all centroids
        sims = centroids.dot(normalized_query)
        nearest_centroids = np.argpartition(sims, -n_probe)[-n_probe:]

        # Clean up memory
        del centroids, sims

        # Load header that tells us where in the flat index file each cluster's indices are
        header_path = os.path.join(self.index_path, "index_header.json")
        with open(header_path, "r") as hf:
            header = json.load(hf)

        # Turn header into a dict for faster lookup: cid -> (offset, length)
        header_dict = {item["cid"]: (item["offset"], item["length"]) for item in header}

        # Prepare top-k heap
        top_heap = []

        flat_index_path = os.path.join(self.index_path, "all_indices.bin")

        # For each centroid to probe:
        for c in nearest_centroids:
            offset, length = header_dict[c]
            if length == 0:
                continue

            # Process each chunk by remapping the memmap for that chunk
            for start in range(0, length, chunk_size):
                cur_len = min(chunk_size, length - start)
                # Remap only this chunk
                ids_mm = np.memmap(flat_index_path, dtype=np.uint32, mode="r",
                                offset=offset + start * np.dtype(np.uint32).itemsize,
                                shape=(cur_len,))

                chunk_ids = ids_mm[:]  # copy if you need to
                del ids_mm  # free memmap

                for id in chunk_ids:
                    vec = self.get_one_row(id)
                    norm = np.linalg.norm(vec)
                    score = vec.dot(normalized_query) / (norm if norm != 0 else 1.0)
                    if len(top_heap) < top_k:
                        heapq.heappush(top_heap, (score, id))
                    else:
                        heapq.heappushpop(top_heap, (score, id))

                    del vec, score
                



                del chunk_ids




        # Extract and return top-k IDs sorted by score
        results = [idx for score, idx in heapq.nlargest(top_k, top_heap)]
        del top_heap
        return results
    def _build_index(self):
       
        # sqrt(N) rule
        self.no_centroids = 250
        # data is a reference to the memmap object, not the data in RAM
        data = self.get_all_rows()

        kmeans = KMeans(
            n_clusters=self.no_centroids,
            init='k-means++',   # <-- Use k-means++ initialization
            random_state=42 
        )

        kmeans.fit(data)

        # labels and centers are new arrays created in RAM
        labels = kmeans.labels_
        centers = kmeans.cluster_centers_.astype(np.float32)

        # Added deletion of the memmap reference and the kmeans object
        del data
        del kmeans


               
        if not os.path.isdir(self.index_path):
            os.makedirs(self.index_path, exist_ok=True)
        # 1. Build up a list of (cluster_id, indices_array)
        cluster_infos = []
        for cid in range(self.no_centroids):
            indices = np.where(labels == cid)[0].astype(np.uint32)
            cluster_infos.append((cid, indices))


        # 2. Write all indices into one big flat file
        flat_path = os.path.join(self.index_path, "all_indices.bin")
        with open(flat_path, "wb") as f:
            offset = 0
            header = []  # list of (cid, offset, length)
            for cid, inds in cluster_infos:
                length = inds.size
                f.write(inds.tobytes())  # or .tofile but with offset tracking
                header.append((cid, offset, length))
                offset += length * inds.dtype.itemsize

        # 3. Write header metadata
        header_path = os.path.join(self.index_path, "index_header.json")

        with open(header_path, "w") as hf:
            json.dump([{"cid": cid, "offset": off, "length": length} for cid, off, length in header], hf)


        norms = np.linalg.norm(centers, axis=1, keepdims=True)
        centers = centers / (norms + 1e-12)
        # 4. Save centers as before
        np.save(os.path.join(self.index_path, "centers.npy"), centers)

            






In [46]:
import numpy as np
import os
# from vec_db import VecDB
import time
from dataclasses import dataclass
from typing import List
from memory_profiler import memory_usage
import gc

@dataclass
class Result:
    run_time: float
    top_k: int
    db_ids: List[int]
    actual_ids: List[int]

def run_queries(db, queries, top_k, actual_ids, num_runs):
    """
    Run queries on the database and record results for each query.

    Parameters:
    - db: Database instance to run queries on.
    - queries: List of query vectors.
    - top_k: Number of top results to retrieve.
    - actual_ids: List of actual results to evaluate accuracy.
    - num_runs: Number of query executions to perform for testing.

    Returns:
    - List of Result
    """
    global results
    results = []
    for i in range(num_runs):
        tic = time.time()
        db_ids = db.retrieve(queries[i], top_k)
        toc = time.time()
        run_time = toc - tic
        results.append(Result(run_time, top_k, db_ids, actual_ids[i]))
    return results

def memory_usage_run_queries(args):
    """
    Run queries and measure memory usage during the execution.

    Parameters:
    - args: Arguments to be passed to the run_queries function.

    Returns:
    - results: The results of the run_queries.
    - memory_diff: The difference in memory usage before and after running the queries.
    """
    global results
    mem_before = max(memory_usage())
    mem = memory_usage(proc=(run_queries, args, {}), interval = 1e-3)
    return results, max(mem) - mem_before

def evaluate_result(results: List[Result]):
    """
    Evaluate the results based on accuracy and runtime.
    Scores are negative. So getting 0 is the best score.

    Parameters:
    - results: A list of Result objects

    Returns:
    - avg_score: The average score across all queries.
    - avg_runtime: The average runtime for all queries.
    """
    scores = []
    run_time = []
    for res in results:
        run_time.append(res.run_time)
        # case for retireving number not equal to top_k, socre will be the lowest
        if len(set(res.db_ids)) != res.top_k or len(res.db_ids) != res.top_k:
            scores.append( -1 * len(res.actual_ids) * res.top_k)
            continue
        score = 0
        for id in res.db_ids:
            try:
                ind = res.actual_ids.index(id)
                if ind > res.top_k * 3:
                    score -= ind
            except:
                score -= len(res.actual_ids)
        scores.append(score)

    return sum(scores) / len(scores), sum(run_time) / len(run_time)

def get_actual_ids_first_k(actual_sorted_ids, k):
    """
    Retrieve the IDs from the sorted list of actual IDs.
    actual IDs has the top_k for the 20 M database but for other databases we have to remove the numbers higher than the max size of the DB.

    Parameters:
    - actual_sorted_ids: A list of lists containing the sorted actual IDs for each query.
    - k: The DB size.

    Returns:
    - List of lists containing the actual IDs for each query for this DB.
    """
    return [[id for id in actual_sorted_ids_one_q if id < k] for actual_sorted_ids_one_q in actual_sorted_ids]

This code to generate all the files for databases.

In [38]:
def _write_vectors_to_file(vectors: np.ndarray, db_path) -> None:
    mmap_vectors = np.memmap(db_path, dtype=np.float32, mode='w+', shape=vectors.shape)
    mmap_vectors[:] = vectors[:]
    mmap_vectors.flush()

# def generate_database(size: int) -> None:
#     rng = np.random.default_rng(DB_SEED_NUMBER)
#     vectors = rng.random((size, DIMENSION), dtype=np.float32)
#     return vectors
def generated_database():
    # Load memmap
    mmap_vectors = np.memmap("data_20M.dat", dtype=np.float32, mode='r', shape=(20_000_000, 64))
    return mmap_vectors


vectors = generated_database()

db_filename_size_20M = 'saved_db_20M.dat'
if not os.path.exists(db_filename_size_20M): _write_vectors_to_file(vectors, db_filename_size_20M)
db_filename_size_15M = 'saved_db_15M.dat'
if not os.path.exists(db_filename_size_15M): _write_vectors_to_file(vectors[:15*10**6], db_filename_size_15M)
db_filename_size_10M = 'saved_db_10M.dat'
if not os.path.exists(db_filename_size_10M): _write_vectors_to_file(vectors[:10*10**6], db_filename_size_10M)
db_filename_size_1M = 'saved_db_1M.dat'
if not os.path.exists(db_filename_size_1M): _write_vectors_to_file(vectors[:1*10**6], db_filename_size_1M)


In [40]:
needed_top_k = 10000
rng = np.random.default_rng(QUERY_SEED_NUMBER)
query1 = rng.random((1, 64), dtype=np.float32)
query2 = rng.random((1, 64), dtype=np.float32)
query3 = rng.random((1, 64), dtype=np.float32)
query_dummy = rng.random((1, 64), dtype=np.float32)

actual_sorted_ids_20m_q1 = np.argsort(vectors.dot(query1.T).T / (np.linalg.norm(vectors, axis=1) * np.linalg.norm(query1)), axis= 1).squeeze().tolist()[::-1][:needed_top_k]
gc.collect()
actual_sorted_ids_20m_q2 = np.argsort(vectors.dot(query2.T).T / (np.linalg.norm(vectors, axis=1) * np.linalg.norm(query2)), axis= 1).squeeze().tolist()[::-1][:needed_top_k]
gc.collect()
actual_sorted_ids_20m_q3 = np.argsort(vectors.dot(query3.T).T / (np.linalg.norm(vectors, axis=1) * np.linalg.norm(query3)), axis= 1).squeeze().tolist()[::-1][:needed_top_k]
gc.collect()

queries = [query1, query2, query3]
actual_sorted_ids_20m = [actual_sorted_ids_20m_q1, actual_sorted_ids_20m_q2, actual_sorted_ids_20m_q3]

C:\Users\m\AppData\Local\Temp\ipykernel_2944\1873110300.py:8: RuntimeWarning: invalid value encountered in divide
  actual_sorted_ids_20m_q1 = np.argsort(vectors.dot(query1.T).T / (np.linalg.norm(vectors, axis=1) * np.linalg.norm(query1)), axis= 1).squeeze().tolist()[::-1][:needed_top_k]
C:\Users\m\AppData\Local\Temp\ipykernel_2944\1873110300.py:10: RuntimeWarning: invalid value encountered in divide
  actual_sorted_ids_20m_q2 = np.argsort(vectors.dot(query2.T).T / (np.linalg.norm(vectors, axis=1) * np.linalg.norm(query2)), axis= 1).squeeze().tolist()[::-1][:needed_top_k]
C:\Users\m\AppData\Local\Temp\ipykernel_2944\1873110300.py:12: RuntimeWarning: invalid value encountered in divide
  actual_sorted_ids_20m_q3 = np.argsort(vectors.dot(query3.T).T / (np.linalg.norm(vectors, axis=1) * np.linalg.norm(query3)), axis= 1).squeeze().tolist()[::-1][:needed_top_k]


In [41]:
# No more need to the actual vectors so delete it
del vectors
gc.collect()

0

In [42]:
results = []
to_print_arr = []

In [10]:
print("Team Number", 3)
database_info = {
    "1M": {
        "database_file_path": db_filename_size_1M,
        "index_file_path": "PATH_DB_1M",
        "size": 10**6
    },
    # "10M": {
    #     "database_file_path": db_filename_size_10M,
    #     "index_file_path": "PATH_DB_10M",
    #     "size": 10 * 10**6
    # },
    # "15M": {
    #     "database_file_path": db_filename_size_15M,
    #     "index_file_path": PATH_DB_15M,
    #     "size": 15 * 10**6
    # },
    # "20M": {
    #     "database_file_path": db_filename_size_20M,
    #     "index_file_path": PATH_DB_20M,
    #     "size": 20 * 10**6
    # }
}


Team Number 3


In [51]:

for db_name, info in database_info.items():
    db = VecDB(database_file_path = info["database_file_path"], index_file_path = info["index_file_path"], new_db = False)
    db._build_index()

In [52]:

for db_name, info in database_info.items():
    db = VecDB(database_file_path = info["database_file_path"], index_file_path = info["index_file_path"], new_db = False)
    actual_ids = get_actual_ids_first_k(actual_sorted_ids_20m, info["size"])
    # Make a dummy run query to make everything fresh and loaded (wrap up)
    res = run_queries(db, query_dummy, 5, actual_ids, 1)
    # actual runs to evaluate
    res, mem = memory_usage_run_queries((db, queries, 5, actual_ids, 3))
    eval = evaluate_result(res)
    to_print = f"{db_name}\tscore\t{eval[0]}\ttime\t{eval[1]:.2f}\tRAM\t{mem:.2f} MB"
    print(to_print)
    to_print_arr.append(to_print)
    del db
    del actual_ids
    del res
    del mem
    del eval
    gc.collect()

1M	score	0.0	time	3.61	RAM	1.62 MB


In [ ]:
print("Team Number", 3)
print("\n".join(to_print_arr))